Train model

In [1]:
# Provides similar functionality to ImageDataGenerators for videos
!pip install keras-video-generators

  Created wheel for keras-video-generators: filename=keras_video_generators-1.0.14-cp37-none-any.whl size=12884 sha256=99ebb18b46c379bd66a3524a61f60d87a83b186bbe25211fe3dc312596df2b34
  Stored in directory: /root/.cache/pip/wheels/20/b7/76/8674d46fc4777c09e5aa7b065d4e356d90f12ec409a6144bbb
Successfully built keras-video-generators


In [2]:
# Please email me at nini16@tamu.edu if you do not ave access to the google drive.
# Permissions should have been granted but if not please email me!

from google.colab import drive
drive.mount('/content/drive/')
# drive.flush_and_unmount()

Mounted at /content/drive/


In [3]:
# training data.
# Please ensure the file is present before running.
!unzip -q "/content/drive/MyDrive/CSCE636/v10/dataset_v10.zip"

In [4]:
import keras
from keras.regularizers import l2
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D, Dense, Dropout, Bidirectional
from keras.preprocessing.image import ImageDataGenerator
from keras_video import VideoFrameGenerator

from keras.layers import TimeDistributed, GRU, Dense, Dropout, LSTM

from keras.models import load_model, Model

from keras.applications import InceptionV3, DenseNet121

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import os
import numpy as np

import math

In [5]:
# All frames should be resized
# Please select a batch-size that divides the number of samples!
# THE DATASET WILL PROBABLY BE INCREASED FOR THE NEXT SUBMISSION SO
# BE SURE TO ADJUST THE BATCH SIZE!!

img_shape = (224, 224)
BS = 22

In [6]:
# Apply image augmentation to each frame
# Please confirm that this directory is present before running

vid_gen = VideoFrameGenerator(
    glob_pattern=r"/content/dataset_v10/{classname}/*",
    nb_frames=20, 
    shuffle=True,
    split_val=0.1155,
    batch_size=BS,
    target_shape=img_shape,
    nb_channel=3,
    transformation=ImageDataGenerator(rescale=1./255,
                                      samplewise_center=True,
                                      # rotation_range=30,
                                      # width_shift_range=0.1,
                                      # height_shift_range=0.1,
                                      # shear_range=0.1,
                                      # zoom_range=0.1,
                                      # horizontal_flip=True,
                                      fill_mode="nearest"),
    use_frame_cache=False)

class watering_plants, validation count: 55, train count: 429
class z_miscellaneous, validation count: 55, train count: 429
Total data: 2 classes for 858 files for train


In [7]:
validation_gen = vid_gen.get_validation_generator()

Total data: 2 classes for 110 files for validation


In [8]:
# model structure for Feature Extractor
def build_convnet_3(shape=(224, 224, 3)):
    incnet = InceptionV3(include_top=False, weights='imagenet', input_shape=shape)

    train = False
    for layer in incnet.layers:
        layer.trainable = train
#         if layer.name == "conv5_block3_2_relu":
#             train = True
    
    globMaxpool = GlobalMaxPool2D()(incnet.output)
    model = Model(inputs=incnet.input, outputs=globMaxpool)
    return model

In [10]:
def action_model(shape=(20,) + img_shape + (3,)):
    # Create our feature extractor convnet with img_shape input shape
    convnet = build_convnet_3()
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with img_shape shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # add GRU
    model.add(Bidirectional(LSTM(64)))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu', kernel_regularizer=keras.regularizers.l2(l2=0.01)))
    model.add(Dropout(.5))
    model.add(Dense(2, activation='softmax'))

    model.summary()
    return model

In [11]:
# instantiate and compile model
model = action_model()
optimizer = keras.optimizers.Adam(0.0005) #0.0005
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

87916544/87910968 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 20, 2048)          21802784  
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               1081856   
_________________________________________________________________
dense (Dense)                (None, 1024)              132096    
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2050      
Total params: 23,018,786
Trainable params: 1,216,002
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
# Adjust epochs and other parameters as needed
# Callbacks have been commented out to avoid overwriting existin data.
# Whoever is running this can uncomment them as needed

callbacks = [
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, min_lr=0.0001),
    keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=5,
        ),
    keras.callbacks.ModelCheckpoint(
        r'C:\Users\cotua\Desktop\python scripts\Weights\weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        monitor='val_acc',
        save_best_only=True,
        verbose=1),
]

history = model.fit_generator(
    vid_gen,
    steps_per_epoch=math.ceil(vid_gen.files_count/BS),
    validation_data=validation_gen,
    verbose=1,
    epochs=30, # last used 80,
    shuffle=True,
    callbacks=callbacks
)

c:\users\cotua\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
39/39 [==============================] - 186s 4s/step - loss: 2.6939 - acc: 0.5431 - val_loss: 1.7272 - val_acc: 0.8727

Epoch 00001: val_acc improved from -inf to 0.87273, saving model to C:\Users\cotua\Desktop\python scripts\Weights\weights.01-1.73.hdf5
Epoch 2/30
39/39 [==============================] - 32s 809ms/step - loss: 1.4729 - acc: 0.8840 - val_loss: 1.0070 - val_acc: 0.9000

Epoch 00002: val_acc improved from 0.87273 to 0.90000, saving model to C:\Users\cotua\Desktop\python scripts\Weights\weights.02-1.01.hdf5
Epoch 3/30
39/39 [==============================] - 32s 811ms/step - loss: 0.8276 - acc: 0.9412 - val_loss: 0.6577 - val_acc: 0.9182

Epoch 00003: val_acc improved from 0.90000 to 0.91818, saving model to C:\Users\cotua\Desktop\python scripts\Weights\weights.03-0.66.hdf5
Epoch 4/30
39/39 [==============================] - 32s 810ms/step - loss: 0.5225 - acc: 0.9762 - val_loss: 0.4917 - val_acc: 0.9455

Epoch 00004: val_acc improved from 0.91818 to 0.94545, 

In [ ]:
# uncomment only if you need to
model.save(r'C:\Users\cotua\Desktop\python scripts\Inception_BLSTM_model_V10_lr_0.0005.h5')

In [ ]:
# uncomment only if you need to
np.save(r'C:\Users\cotua\Desktop\python scripts\train_history_Inception_model_V10_lr_0.0005.npy',history.history)

**Testing model**

In [ ]:
# Note that this takes a lot of RAM and might crash. Hence, reduce step size to run

In [13]:
model=load_model(r'/content/drive/MyDrive/CSCE636/v10/Inception_BLSTM_model_V10_lr_0.0005.h5')

In [17]:
!unzip -q "/content/drive/MyDrive/CSCE636/v5/youtube-neg-test.zip"

In [15]:
!unzip -q "/content/drive/MyDrive/CSCE636/v10/youtube-pos-test.zip"

Positive Youtube Videos

In [16]:
test_gen = VideoFrameGenerator(
    glob_pattern=r"/content/youtube-pos-test/{classname}/*",
    nb_frames=20, 
    shuffle=False,
    batch_size=16,
    target_shape=img_shape,
    nb_channel=3,
    transformation=ImageDataGenerator(rescale=1./255,
                                      samplewise_center=True,
                                      # rotation_range=30,
                                      # width_shift_range=0.1,
                                      # height_shift_range=0.1,
                                      # shear_range=0.1,
                                      # zoom_range=0.1,
                                      # horizontal_flip=True,
                                      fill_mode="nearest"),
    use_frame_cache=False)

Total data: 2 classes for 272 files for train


In [ ]:
len(test_gen)

17

In [ ]:
model.evaluate_generator(test_gen, steps=len(test_gen))

c:\users\cotua\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.23335778713226318, 0.9485294222831726]

Negative Youtube Videos

In [18]:
test_gen = VideoFrameGenerator(
    glob_pattern=r"/content/youtube-neg-test/{classname}/*",
    nb_frames=20, 
    shuffle=True,
    batch_size=31,
    target_shape=img_shape,
    nb_channel=3,
    transformation=ImageDataGenerator(rescale=1./255,
                                      samplewise_center=True,
                                      # rotation_range=30,
                                      # width_shift_range=0.1,
                                      # height_shift_range=0.1,
                                      # shear_range=0.1,
                                      # zoom_range=0.1,
                                      # horizontal_flip=True,
                                      fill_mode="nearest"),
    use_frame_cache=False)

Total data: 2 classes for 310 files for train


In [ ]:
len(test_gen)

10

In [20]:
model.evaluate_generator(test_gen, steps=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.5868940949440002, 0.9032257795333862]